In [ ]:
import coolingFunction

In [ ]:
import copy
import numpy as np
import h5py
import scipy
import scipy.special
import sys
import verdict
import os
import tqdm
import unyt

In [ ]:
import kalepy as kale

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.patheffects as path_effects
import matplotlib.cm as cm
import matplotlib.colors as plt_colors
import matplotlib.gridspec as gridspec
import matplotlib.transforms
import palettable

In [ ]:
import linefinder.analyze_data.worldlines as a_worldlines
import linefinder.analyze_data.worldline_set as worldline_set
import linefinder.analyze_data.plot_worldlines as p_worldlines
import linefinder.utils.presentation_constants as p_constants

In [ ]:
import galaxy_dive.analyze_data.ahf as analyze_ahf
import galaxy_dive.plot_data.ahf as plot_ahf
import galaxy_dive.analyze_data.particle_data as particle_data
import galaxy_dive.plot_data.generic_plotter as generic_plotter
import galaxy_dive.plot_data.plotting as plotting
import galaxy_dive.utils.data_operations as data_operations
import galaxy_dive.utils.executable_helpers as exec_helpers

In [ ]:
import linefinder.utils.file_management as file_management
import linefinder.config as config

In [ ]:
import trove

In [ ]:
import helpers

# Load Data

In [ ]:
pm = dict(
    snum = 600,
    tables_dir = '/work/03057/zhafen/CoolingTables/',
    study_duplicates = False,
    ahf_index = 600,
    
#     # If we want to ensure some minimum number of snapshots in the galaxy after accreting
#     # (remember to account for the last 10 snapshots with small dt)
#     minInd = 0,
#     maxInd = 54, # Corresponds to 1 Gyr. We don't look at accretion prior.
    
    # For the fancy scatter plot we're visualizing.
    variable_alpha = True,
)

In [ ]:
pm = trove.link_params_to_config(
    '/home1/03057/zhafen/papers/Hot-Accretion-in-FIRE/analysis/hot_accretion.trove',
    script_id = 'nb.10',
    **pm
)

In [ ]:
# Used so often it's nice to not enclose
snum = pm['snum']
ind = pm['ahf_index'] - snum

In [ ]:
w = a_worldlines.Worldlines(
    tag = pm['tag'],
    data_dir = pm['base_data_dir'],
    halo_data_dir = pm['halo_data_dir'],
    ahf_index = pm['ahf_index'],
)

In [ ]:
w.retrieve_halo_data()

In [ ]:
m_plot_label  = r'$M_{\rm h} = 10^{' + '{:.02g}'.format( np.log10( w.m_vir[snum] ) )
m_plot_label += '} M_\odot$'
plot_label = m_plot_label + ', z={:.02}'.format( w.redshift[snum] )
print( plot_label )

In [ ]:
classification_list = copy.copy( p_constants.CLASSIFICATIONS_CGM_FATE )

In [ ]:
w_plotter = p_worldlines.WorldlinesPlotter( w, label=plot_label )

In [ ]:
base_processed_data_dir = pm['config_parser'].get( 'DEFAULT', 'processed_data_dir' )
default_data_fp = os.path.join( base_processed_data_dir, 'summary.hdf5' )
default_data = verdict.Dict.from_hdf5( default_data_fp, create_nonexistent=True )

In [ ]:
data_fp = os.path.join( pm['processed_data_dir'], 'summary.hdf5' )
data = verdict.Dict.from_hdf5( data_fp, create_nonexistent=True )

## Labels

In [ ]:
tchange_key = pm['central_indices'].split( '_' )[0]
t_tchange_key = 't_' + tchange_key

In [ ]:
t_tchange_label = helpers.get_t_tchange_label( pm )

# Data Pre-Processing

## Calculate $\theta$
Also called $\phi$...

In [ ]:
tot_momentum_fp = os.path.join( base_processed_data_dir, 'tot_momentums.hdf5' )
tot_ang_momentum = verdict.Dict.from_hdf5( tot_momentum_fp )[pm['variation']]['snum{:03d}'.format( snum )]

In [ ]:
w.total_ang_momentum = tot_ang_momentum

In [ ]:
w.calc_abs_phi()

## Calculate $\vec j$

In [ ]:
specific_mom = w.get_data( 'J' )
w.data['Jmag'] = w.get_data( 'Jmag' )

In [ ]:
tot_ang_momentum_normed = tot_ang_momentum / np.linalg.norm( tot_ang_momentum )
w.data['Jz'] = np.array([ np.dot( specific_mom[:,:,i].transpose(), tot_ang_momentum_normed ) for i in range( w.snums.size ) ]).transpose()

In [ ]:
w.data['Jz/J'] = w.data['Jz'] / w.data['Jmag']

## Calculate central indices

In [ ]:
if pm['central_indices'] == 'tcools_inds':
    inds = w.calc_tcools_inds(
        lookback_time_max = pm['lookback_time_max'],
        choose_first = pm['choose_first'],
        B = pm['logTcools'],
    )
else:
    calc_fn = getattr( w, 'calc_{}'.format( pm['central_indices'] ) )
    inds = calc_fn(
        lookback_time_max = pm['lookback_time_max'],
        choose_first = pm['choose_first'],
    )

# Analysis

In [ ]:
# Setup axes
t_window = 1.
t = w.get_data( 'time' )
x_range = [ t[ind] - t_window, t[ind] ]

In [ ]:
t_snaps = t[( t > x_range[0] ) & ( t < x_range[1] )][::-1]

In [ ]:
n_snaps = t_snaps.size

In [ ]:
dt = t_snaps[1:] - t_snaps[:-1]

In [ ]:
t_bins = np.zeros( ( t_snaps.size + 1, ) )
t_bins[1:-1] = t_snaps[:-1] + dt / 2.
t_bins[0] = t_snaps[0] - dt[0] / 2.
t_bins[-1] = t_snaps[-1] + dt[-1] / 2.

In [ ]:
   
w.data_masker.clear_masks()
w.data_masker.mask_data( 'PType', data_value=0 )

# Median and interval stats
logT = np.log10( w.get_selected_data( 'T', compress=False ) )#[:,ind:ind+n_snaps+1]
R = w.get_selected_data( 'R', compress=False )#[:,ind:ind+n_snaps+1]
L = w.get_selected_data( 'Lmag', compress=False )#[:,ind:ind+n_snaps+1]
M = w.get_selected_data( 'M', compress=False )#[:,ind:ind+n_snaps+1]

logT_med = np.nanmedian( logT, axis=0 )
R_med = np.nanmedian( R, axis=0 )

logT_low = np.nanpercentile( logT, 16, axis=0 )
logT_high = np.nanpercentile( logT, 84, axis=0 )

R_low = np.nanpercentile( R, 16, axis=0 )
R_high = np.nanpercentile( R, 84, axis=0 )

inds = w.get_data( pm['central_indices'] )
particle_inds = np.arange( inds.size )

valid = ( inds >= pm['minInd'] ) & ( inds < pm['maxInd'] )
R_at_tchange = R[particle_inds[valid],inds[valid]]
M_at_tchange = M[particle_inds[valid],inds[valid]]
L_at_tchange = R[particle_inds[valid],inds[valid]]
R_rgal_at_tchange = R[particle_inds[valid],inds[valid]] / w.r_gal[inds[valid]]
t_at_tchange = t[inds[valid]]

# R_at_tchange = np.array( [ R[i, ind] for i, ind in enumerate( inds ) ] )[inds >= pm['minInd']]
# M_at_tchange = np.array( [ M[i, ind] for i, ind in enumerate( inds ) ] )[inds >= pm['minInd']]
# L_at_tchange = np.array( [ L[i, ind] for i, ind in enumerate( inds ) ] )[inds >= pm['minInd']]
# R_rgal_at_tchange = np.array( [ R[i, ind]/w.r_gal[ind] for i, ind in enumerate( inds ) ] )[inds >= pm['minInd']]

# t_at_tchange = np.array( [ t[ind] for ind in inds ] )[inds >= pm['minInd']]


## Store for Later Use

In [ ]:
# Store R for later use
r_vir = w.r_vir[snum]
r_points, r_pdf = kale.density(
    R_at_tchange[np.invert(np.isnan(R_at_tchange))],
    points = np.linspace( 0., r_vir, 512 ),
    probability = True,
    reflect = [ 0., None ],
)

In [ ]:
data_to_store = {
    'points': r_points,
    'pdf': r_pdf,
    'median': np.nanmedian( R_at_tchange ),
    '16th_percentile': np.nanpercentile( R_at_tchange, 16 ),
    '84th_percentile': np.nanpercentile( R_at_tchange, 84 ),
}

In [ ]:
# Store radius for later use
flag = pm['central_indices'].split( '_' )[0][1:]
r_key = 'R' + flag
if not r_key in data.keys():
    data[r_key] = {}
for key, item in data_to_store.items():
    if key not in data[r_key]:
        data[r_key][key] = {}
    data[r_key][key][pm['variation']] = item

In [ ]:
# Store R for later use
r_vir = w.r_vir[snum]
r_points, r_pdf = kale.density(
    R_rgal_at_tchange[np.invert(np.isnan(R_rgal_at_tchange))],
    points = np.linspace( 0., r_vir / w.r_gal[0], 512 ),
    probability = True,
    reflect = [ 0., None ],
)

In [ ]:
data_to_store = {
    'points': r_points,
    'pdf': r_pdf,
    'median': np.nanmedian( R_rgal_at_tchange ),
    '16th_percentile': np.nanpercentile( R_rgal_at_tchange, 16 ),
    '84th_percentile': np.nanpercentile( R_rgal_at_tchange, 84 ),
}

In [ ]:
# Store radius relative to galaxy radius for later use
rrgal_key = r_key + '_rgal'
if not rrgal_key in data.keys():
    data[rrgal_key] = {}
for key, item in data_to_store.items():
    if key not in data[rrgal_key]:
        data[rrgal_key][key] = {}
    data[rrgal_key][key][pm['variation']] = item

In [ ]:
# Store number of particles tracked
if not 'n_tracked' in data.keys():
    data['n_tracked'] = {}
data['n_tracked'][pm['variation']] = w.n_particles

In [ ]:
data.to_hdf5( data_fp, handle_jagged_arrs='row datasets' )
print( 'Stored summary data at {}'.format( data_fp ) )

## Angular Distribution

In [ ]:
bins = np.linspace( -1., 1., 128 )
centers = bins[:-1] + 0.5 * ( bins[1] - bins[0] )

In [ ]:
dt = 0.1
t_tchange_centers = np.arange( -1.0, 0.5 + 0.01, 0.01 )

In [ ]:
def calc_sph_harm_moment( l, m, phi, masses, radii ):
    
    prefactor = np.sqrt( 4. * np.pi / (2. * l + 1) )
    ylm = scipy.special.sph_harm( m, l, 0., phi ).real
    to_sum = masses * radii**l * ylm
    
    return prefactor * to_sum.sum()

In [ ]:
# Get the time at the phase
t_tchange = w.get_data( t_tchange_key )
t_tchange_flat = t_tchange.flatten()
t_post_inds = np.argmin( np.abs( t_tchange - 0.150 ), axis=1 )
t_pre_inds = np.argmin( np.abs( t_tchange + 0.150 ), axis=1 )
insufficient_time_after = t_post_inds == 0
insufficient_time_after_mask = np.tile( insufficient_time_after, ( w.n_snaps, 1 ) ).transpose()

In [ ]:
# Mask Data
w.data_masker.clear_masks()
w.data_masker.mask_data( 'PType', data_value=0 )
w.data_masker.mask_data( 'insufficient_time_after', custom_mask=insufficient_time_after_mask,  )

In [ ]:
# Get data
phi = w.get_selected_data( 'Phi', compress=False ).flatten()
r_scale = np.full( w.n_snaps, np.nan )
r_scale[:w.r_gal.size] = w.r_gal
radii = ( w.get_selected_data( 'R', compress=False ) / r_scale ).flatten()
masses = w.get_selected_data( 'M', compress=False ).flatten()

In [ ]:
# Get distributions
cosphi_dists = []
cosphi_pdfs = []
cosphi_16ths = []
cosphi_84ths = []
cosphi_stds = []
q20 = []
q33 = []
for i, center in enumerate( tqdm.tqdm( t_tchange_centers ) ):
    bin_low = center - dt / 2.
    bin_high = center + dt / 2.
    in_bin = ( t_tchange_flat > bin_low ) & ( t_tchange_flat < bin_high )
    
    phi_arr = phi[in_bin]
    phi_arr_rad = phi_arr * np.pi / 180.
    cosphi_arr = np.cos( phi_arr_rad )
    cosphi_points, cosphi_pdf = kale.density(
        cosphi_arr[np.invert(np.isnan(cosphi_arr))],
        points = centers,
        probability = True,
        reflect = [ -1., 1. ],
    )
    cosphi_hist, cosphi_bins = np.histogram(
        cosphi_arr[np.invert(np.isnan(cosphi_arr))],
        bins = bins,
        density = True,
    )
    cosphi_pdfs.append( cosphi_pdf )
    cosphi_dists.append( cosphi_hist )
    
    cosphi_arr_comp = cosphi_arr.compressed()
    cosphi_16ths.append( np.nanpercentile( cosphi_arr_comp, 16 ) )
    cosphi_84ths.append( np.nanpercentile( cosphi_arr_comp, 84. ) )
    cosphi_stds.append( np.nanstd( cosphi_arr_comp ) )
    
    masked = np.invert( radii.mask ) & in_bin
    q20_i = calc_sph_harm_moment(
        2,
        0,
        phi[masked].compressed(),
        masses[masked].compressed(),
        radii[masked].compressed()
    )
    q20.append( q20_i )
    q33_i = calc_sph_harm_moment(
        3,
        3,
        phi[masked].compressed(),
        masses[masked].compressed(),
        radii[masked].compressed()
    )
    q33.append( q33_i )

In [ ]:
labeled_is = np.arange( 0, 150, 10 )

In [ ]:
fig = plt.figure( figsize=(10, 4.5 ), facecolor='w' )
ax = plt.gca()

z_max = t_tchange_centers.max()
z_min = t_tchange_centers.min()
        
for i, cosphi_dist in enumerate( cosphi_dists ):

    z_width = z_max - z_min
    color_value = ( t_tchange_centers[i] - z_min )/z_width
    color = palettable.scientific.diverging.Roma_3.mpl_colormap( color_value )

    if i in labeled_is:
        if np.isclose( t_tchange_centers[i], 0. ):
            t_tchange_centers[i] = 0
        label = (
            '{:.3g}'.format( t_tchange_centers[i]*1e3 ) +
            r' Myr'
        )
    else:
        continue

    line = ax.plot(
        centers,
        cosphi_dist, #/ (np.pi / 180. / 2. * np.sin( bin_centers * np.pi/180. ) ),
        linewidth = 1.5,
        color = color,
        label = label,
#             zorder = 10 - i,
    )

ax.tick_params(
    axis = 'x',
    top = True,
    labeltop = ax.is_first_row(),
    bottom = ax.is_last_row(),
    labelbottom = ax.is_last_row(),
)

ax.axhline(
    0.5,
    color = '.2',
    linestyle = '-',
    linewidth = 2,
)
ax.axvline(
    0,
    color = '.2',
    linestyle = '-',
    linewidth = 2,
)

# Sim name label
ax.annotate(
    text = pm['variation'],
    xy = ( 0, 1 ),
    xycoords = 'axes fraction',
    xytext = ( 20, -20 ),
    textcoords = 'offset points',
    ha = 'left',
    va = 'top',
    fontsize = 26,
)

# line labels
ax.annotate(
    text = 'spherical\ndistribution',
    xy = ( -1, 0.5 ),
    xycoords = 'data',
    xytext = ( 10, 10 ),
    textcoords = 'offset points',
    ha = 'left',
    va = 'bottom',
    fontsize = 22,
)
ax.annotate(
    text = 'disc\ndistribution',
    xy = ( 0, 3.75 ),
    xycoords = 'data',
    xytext = ( 15, -10 ),
    textcoords = 'offset points',
    ha = 'left',
    va = 'top',
    fontsize = 22,
)

t_label = ax.annotate(
    text = t_tchange_label,
    xy = ( 1, 0.875 ),
    xycoords = 'axes fraction',
    xytext = ( -25, 0 ),
    textcoords = 'offset points',
    ha = 'right',
    va = 'bottom',
    fontsize = 24,
)
t_label.set_zorder( 1000 )
ax.legend(
    prop={'size': 17},
    loc = 'center right',
)

ax.set_xlim( -1, 1 )
ax.set_ylim( 0, 3.75 )

ax.set_xlabel( r'$\cos\ \theta$', fontsize=22 )
# if ax.is_first_row():
#     ax.xaxis.set_label_position( 'top' )
ax.set_ylabel( r'PDF$\ (\cos\ \theta$)', fontsize=22 )

plotting.save_fig(
    out_dir = os.path.join( pm['figure_dir'], 'ang_dist_evolution' ),
    save_file = 'theta_vs_t_{}.pdf'.format( pm['variation'] ),
    fig = fig,
)

### Store phis

In [ ]:
zero_ind = np.argmin( np.abs( centers ) )
pdf_at_zero = np.array( cosphi_dists )[:,zero_ind]

In [ ]:
data_to_store = {
    'points': centers,
    't_tchange_centers': t_tchange_centers,
    'pdf': np.array( cosphi_pdfs ),
    'hist': np.array( cosphi_dists ),
    '16th_percentile': cosphi_16ths,
    '84th_percentile': cosphi_84ths,
    'std': cosphi_stds,
    'pdf(cos theta=0)': pdf_at_zero,
    'q20': q20,
    'q33': q33,
}

In [ ]:
# Store circularity for later use
if not 'cosphi' in data.keys():
    data['cosphi'] = {}
for key, item in data_to_store.items():
    if key not in data['cosphi']:
        data['cosphi'][key] = {}
    data['cosphi'][key][pm['variation']] = item
data.to_hdf5( data_fp, handle_jagged_arrs='row datasets' )
print( 'Stored summary data at {}'.format( data_fp ) )

## Angular Distribution for Momentum

In [ ]:
dt = 0.1
t_tchange_centers = np.arange( -1.0, 0.5 + 0.01, 0.01 )

In [ ]:
# Mask Data
w.data_masker.clear_masks()
w.data_masker.mask_data( 'PType', data_value=0 )
w.data_masker.mask_data( 'insufficient_time_after', custom_mask=insufficient_time_after_mask,  )

In [ ]:
# Calculate angular momentum
ang_mom_dir = tot_ang_momentum / np.linalg.norm( tot_ang_momentum )
j = w.get_selected_data( 'L', )
jz = np.dot( j.transpose(), ang_mom_dir )
jmag = w.get_selected_data( 'Lmag', )
jz_jmag = ( jz / jmag )

In [ ]:
# Get data
phi = np.arccos( jz_jmag ) * 180. / np.pi
r_scale = np.full( w.n_snaps, np.nan )
r_scale[:w.r_gal.size] = w.r_gal
radii = ( w.get_selected_data( 'R', compress=False ) / r_scale ).compressed()
masses = w.get_selected_data( 'M', )

In [ ]:
# Get masking t_tchange
t_tchange_flat = t_tchange[np.invert( w.data_masker.get_mask() )].flatten()

In [ ]:
# Get distributions
cosphi_dists = []
cosphi_pdfs = []
cosphi_16ths = []
cosphi_84ths = []
cosphi_stds = []
for i, center in enumerate( tqdm.tqdm( t_tchange_centers ) ):
    bin_low = center - dt / 2.
    bin_high = center + dt / 2.
    in_bin = ( t_tchange_flat > bin_low ) & ( t_tchange_flat < bin_high )
    
    if in_bin.sum() == 0:
        cosphi_pdfs.append( np.zeros( centers.shape ) )
        cosphi_dists.append( np.zeros( centers.shape ) )
        cosphi_16ths.append( np.nan )
        cosphi_84ths.append( np.nan )
        cosphi_stds.append( np.nan )
        continue
    
    phi_arr = phi[in_bin]
    phi_arr_rad = phi_arr * np.pi / 180.
    cosphi_arr = np.cos( phi_arr_rad )
    cosphi_points, cosphi_pdf = kale.density(
        cosphi_arr[np.invert(np.isnan(cosphi_arr))],
        points = centers,
        probability = True,
        reflect = [ -1., 1. ],
    )
    cosphi_hist, cosphi_bins = np.histogram(
        cosphi_arr[np.invert(np.isnan(cosphi_arr))],
        bins = bins,
        density = True,
    )
    cosphi_pdfs.append( cosphi_pdf )
    cosphi_dists.append( cosphi_hist )
    
    cosphi_16ths.append( np.nanpercentile( cosphi_arr, 16 ) )
    cosphi_84ths.append( np.nanpercentile( cosphi_arr, 84. ) )
    cosphi_stds.append( np.nanstd( cosphi_arr ) )

In [ ]:
labeled_is = []
for i in range( len( cosphi_dists ) ):
    if i % 20 == 0:
        labeled_is.append( i )

In [ ]:
fig = plt.figure( figsize=(10, 4.5 ), facecolor='w' )
ax = plt.gca()

# z_max = t_tchange_centers.max()
z_min = -0.5
z_max = -z_min
        
for i, cosphi_dist in enumerate( cosphi_dists ):

    z_width = z_max - z_min
    color_value = ( t_tchange_centers[i] - z_min )/z_width
    color = palettable.scientific.diverging.Roma_3.mpl_colormap( color_value )
    
    if ( t_tchange_centers[i] < -0.5 ) or ( t_tchange_centers[i] > 0.50 ):
        continue
        
    if i in labeled_is:
        if np.isclose( t_tchange_centers[i], 0. ):
            t_tchange_centers[i] = 0
        label = (
            '{:.3g}'.format( t_tchange_centers[i]*1e3 ) +
            r' Myr'
        )
    else:
        continue

    cosphi_dist = np.cumsum( cosphi_dist )
    cosphi_dist /= cosphi_dist[-1]
    line = ax.plot(
        centers,
        cosphi_dist, #/ (np.pi / 180. / 2. * np.sin( bin_centers * np.pi/180. ) ),
        linewidth = 5,
        color = color,
        label = label,
#             zorder = 10 - i,
    )

ax.tick_params(
    axis = 'x',
    top = True,
    labeltop = ax.is_first_row(),
    bottom = ax.is_last_row(),
    labelbottom = ax.is_last_row(),
)

ax.axhline(
    0.5,
    color = '.2',
    linestyle = '-',
    linewidth = 2,
)
ax.axvline(
    0,
    color = '.2',
    linestyle = '-',
    linewidth = 2,
)

# Sim name label
ax.annotate(
    text = pm['variation'],
    xy = ( 0, 1 ),
    xycoords = 'axes fraction',
    xytext = ( 20, -20 ),
    textcoords = 'offset points',
    ha = 'left',
    va = 'top',
    fontsize = 26,
)

# line labels
# ax.annotate(
#     text = 'spherical\ndistribution',
#     xy = ( -1, 0.5 ),
#     xycoords = 'data',
#     xytext = ( 10, 10 ),
#     textcoords = 'offset points',
#     ha = 'left',
#     va = 'bottom',
#     fontsize = 22,
# )
# ax.annotate(
#     text = 'disc\ndistribution',
#     xy = ( 0, 3.75 ),
#     xycoords = 'data',
#     xytext = ( 15, -10 ),
#     textcoords = 'offset points',
#     ha = 'left',
#     va = 'top',
#     fontsize = 22,
# )

t_label = ax.annotate(
    text = t_tchange_label,
    xy = ( 1, 0.875 ),
    xycoords = 'axes fraction',
    xytext = ( -25, 0 ),
    textcoords = 'offset points',
    ha = 'right',
    va = 'bottom',
    fontsize = 24,
)
t_label.set_zorder( 1000 )
ax.legend(
    prop={'size': 17},
    bbox_to_anchor = [ 0.5, 1. ],
    loc = 'upper right',
)

# ax.set_xlim( -1, 1 )
ax.set_xlim( 0.5, 1 )
ax.set_ylim( 0, 1 )
# ax.set_ylim( 0.01, 45 )
# ax.set_yscale( 'log' )

ax.set_xlabel( r'$\cos\ \theta$', fontsize=22 )
# if ax.is_first_row():
#     ax.xaxis.set_label_position( 'top' )
ax.set_ylabel( r'PDF$\ (\cos\ \theta$)', fontsize=22 )

plotting.save_fig(
    out_dir = os.path.join( pm['figure_dir'], 'ang_dist_evolution' ),
    save_file = 'jzjmag_vs_t_{}.pdf'.format( pm['variation'] ),
    fig = fig,
)

### Store phis

In [ ]:
zero_ind = np.argmin( np.abs( centers ) )
pdf_at_zero = np.array( cosphi_dists )[:,zero_ind]

In [ ]:
data_to_store = {
    'points': centers,
    't_tchange_centers': t_tchange_centers,
    'pdf': np.array( cosphi_pdfs ),
    'hist': np.array( cosphi_dists ),
    '16th_percentile': cosphi_16ths,
    '84th_percentile': cosphi_84ths,
    'std': cosphi_stds,
    'pdf(cos theta=0)': pdf_at_zero,
}

In [ ]:
# Store circularity for later use
if not 'jzjmag' in data.keys():
    data['jzjmag'] = {}
for key, item in data_to_store.items():
    if key not in data['jzjmag']:
        data['jzjmag'][key] = {}
    data['jzjmag'][key][pm['variation']] = item
data.to_hdf5( data_fp, handle_jagged_arrs='row datasets' )
print( 'Stored summary data at {}'.format( data_fp ) )